# DP Divergence Estimation Tutorial


## _Problem Statement_

When evaluating new testing data, or comparing two datasets, we often want to have a quantitative way of comparing and evaluating shifts in covariates. DP divergence is a nonparametric divergence metric which gives the distance between two datasets. A DP divergence of 0 means that the two datasets are approximately identically distributed. A DP divergence of 1 means the two datasets are completely separable.


### _When to use_

The `Divergence` class should be used when you would like to know how far two datasets are diverged for one another. For example, if you would like to measure operational drift.


### _What you will need_

1. A set of image embeddings for each dataset (usually obtained with an AutoEncoder)


### _Setting up_

Let's import the required libraries needed to set up a minimal working example


In [ ]:
try:
    import google.colab  # noqa: F401

    %pip install -q daml
except Exception:
    pass

import os

from pytest import approx

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

In [ ]:
import numpy as np
import tensorflow_datasets as tfds

from daml.metrics import Divergence

## Loading in data

Let's start by loading in tensorflow's MNIST dataset,
then we will examine it.


In [ ]:
# Load in the mnist dataset from tensorflow datasets
images, ds_info = tfds.load(
    "mnist",
    split="train",
    with_info=True,
)
tfds.visualization.show_examples(images, ds_info)
images = images.shuffle(images.cardinality())
images = [i["image"].numpy() for i in list(images.take(4000))]
images = np.array(images)

In [ ]:
print("Number of samples: ", len(images))
print("Image shape:", images[0].shape)

## Calculate initial divergence

Let's calculate the DP divergence between the first 2500 images and the second 2500 images from this sample.


In [ ]:
data_a = images[0:2000].reshape((2000, -1))
data_b = images[2000:].reshape((2000, -1))

In [ ]:
metric = Divergence(data_a=data_a, data_b=data_b)
div = metric.evaluate()
print(div)

We estimate that the DP divergence between these (identically distributed) images sets is at or close to 0.


## Loading in corrupted data

Now let's load in a corrupted mnist dataset.


In [ ]:
corrupted, ds_info = tfds.load(
    "mnist_corrupted/translate",
    split="train",
    with_info=True,
)
tfds.visualization.show_examples(corrupted, ds_info)
corrupted = corrupted.shuffle(corrupted.cardinality())
corrupted = [i["image"].numpy() for i in list(corrupted.take(2000))]
corrupted = np.array(corrupted)

In [ ]:
print("Number of corrupted samples: ", len(corrupted))
print("Corrupted image shape:", corrupted[0].shape)

## Calculate corrupted divergence

Now lets calculate the DP Divergence between this corrupted dataset and the original images


In [ ]:
metric.data_b = corrupted.reshape((2000, -1))
div = metric.evaluate()
print(div)

In [ ]:
### TEST ASSERTION ###
print(div)
assert div["dpdivergence"] == approx(0.96, abs=0.02)

We conclude that the translated MNIST images are significantly different from the original images.
